# Testing Prompts for generating synthetic data

## Importing libraries

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_nvidia_ai_endpoints import ChatNVIDIA

import os
import pandas as pd

# Set Pandas to display all columns
pd.set_option('display.max_columns', None)

In [2]:
import os
import sys
from pathlib import Path

path = Path(os.path.dirname(os.getcwd()))
path = str(path)
print(path)
sys.path.insert(1, path)

/Users/saideepbunny/Projects/Application_Ranking_System


## Reading the data

In [3]:
df = pd.read_csv(f"{path}/data/kaggle_linkedin_jd_skills_dataset/sampled_jd_skills.csv")
df

,job_link,last_processed_time,got_summary,got_ner,is_being_worked,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type,job_skills,job_summary,id,summary_len,skill_count
0,https://www.linkedin.com/jobs/view/software-ar...,2024-01-19 09:45:09.215838+00,t,t,f,Software Architect,Harris Computer,"Florida, United States",2024-01-13,Beverly Hills,United States,Director Technical,Mid senior,Onsite,"Product architecture, Engineering processes, C...",As Architect you will define the products’ hig...,340370,6407,33
1,https://www.linkedin.com/jobs/view/senior-syst...,2024-01-19 09:45:09.215838+00,t,t,f,Senior System Engineer,ExLabs,"Long Beach, CA",2024-01-13,Compton,United States,Maintenance Data Analyst,Mid senior,Onsite,"Systems engineering, Technical oversight, Requ...",Job Title: Sr. System Engineer\nCompany: ExLab...,640026,6308,32
2,https://www.linkedin.com/jobs/view/senior-data...,2024-01-20 11:15:59.804413+00,t,t,f,Senior Data Scientist,Workforce Connections,"St Louis, MO",2024-01-14,Belleville,United States,Chemist,Mid senior,Onsite,"Data Science, Data Analysis, Machine Learning,...","Location: Onsite, St. Louis MO 63146\nJob Resp...",1096469,3635,30
3,https://www.linkedin.com/jobs/view/distinguish...,2024-01-19 22:25:24.801979+00,t,t,f,Distinguished Applied Researcher,Jobs for Humanity,"Fort Worth, TX",2024-01-15,Fort Worth,United States,Director Research,Mid senior,Onsite,"AI, Applied Research, Machine Learning, Natura...",Company Description\nJobs for Humanity is part...,998553,4011,34
4,https://www.linkedin.com/jobs/view/systems-adm...,2024-01-21 01:06:09.90334+00,t,t,f,Systems Administrator,Leidos,"Colorado Springs, CO",2024-01-14,Cripple Creek,United States,Tier,Mid senior,Onsite,"System Administration, IT Operating Systems, C...",Description\nThe Leidos Digital Modernization ...,1189859,4117,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,https://uk.linkedin.com/jobs/view/workshop-con...,2024-01-19 09:45:09.215838+00,t,t,f,Workshop Controller,Activate Group Limited,"Bury St. Edmunds, England, United Kingdom",2024-01-13,Ipswich,United Kingdom,Controller,Mid senior,Onsite,"Workshop Control, Body Shop Management, HighQu...",Job title: Workshop Controller\nDepartment: Ac...,321651,4703,32
1496,https://www.linkedin.com/jobs/view/x-ray-techn...,2024-01-21 04:16:54.473867+00,t,t,f,X-Ray Technologist,Fast Pace Health,"Grant, AL",2024-01-17,Guntersville,United States,Ultrasound Technologist,Mid senior,Onsite,"XRay Technologist, HIPAA, Patient intake, Nurs...",Posting Title: X-Ray Technologist Overview:\nF...,1230454,5704,38
1497,https://www.linkedin.com/jobs/view/xfinity-ret...,2024-01-19 23:17:04.320124+00,t,t,f,Xfinity Retail Sales Consultant,XFINITY Store by Comcast,"East Brunswick, NJ",2024-01-14,Long Branch,United States,Sales-Service Promoter,Mid senior,Onsite,"Customer service, Product demonstration, Probl...",Job Summary\nResponsible for assisting and con...,159258,4839,32
1498,https://www.linkedin.com/jobs/view/yard-associ...,2024-01-19 09:45:09.215838+00,t,t,f,Yard Associate (SkillBridge),ARMOR Initiative,"Kansas City, MO",2024-01-13,Leavenworth,United States,Orderly,Associate,Onsite,"DoD SkillBridge Internship, Fulltime training,...",The ARMOR Initiative DoD SkillBridge Internshi...,774643,2277,30


## Testing prompts

In [24]:
i=183

In [25]:
print("**Job role:**", df.iloc[i]["job_title"])
print("\n**Job Description:**", df.iloc[i]["job_summary"])
print("\n**Skills:**", df.iloc[i]["job_skills"])


**Job role:** Experienced Associate, Software Engineer (Python/SQL)

**Job Description:** Specialty/Competency:
Risk
Industry/Sector:
Not Applicable
Time Type:
Full time
Travel Requirements:
Up to 60%
A career in our Advisory Acceleration Centre is the natural extension of PwC’s leading class global delivery capabilities. We provide premium, cost effective, high quality services that support process quality and delivery capability in support for client engagements.
To really stand out and make us fit for the future in a constantly changing world, each and every one of us at PwC needs to be a purpose-led and values-driven leader at every level. To help us achieve this we have the PwC Professional; our global leadership development framework. It gives us a single set of expectations across our lines, geographies and career paths, and provides transparency on the skills we need as individuals to be successful and progress in our careers, now and in the future.
Responsibilities
As an Assoc

### Generating Resume

In [6]:
category_requirements = {
    "Complete Mismatch": """
    The resume is highly professional but entirely unsuitable for the job. There is no meaningful alignment between the candidate's background and the job requirements.

    - **Career Path:**  
      - The candidate has spent their career in an industry that is entirely unrelated to the job.  
      - Example: A healthcare professional applying for a software engineering role without any technical experience.  

    - **Skills:**  
      - There is no significant overlap between the candidate’s skills and the job requirements.  
      - The candidate specializes in an entirely different domain and lacks both fundamental and advanced job-related skills.  
      - Example: A mechanical engineer with expertise in CAD software applying for a data science role requiring Python and ML expertise.  

    - **Experience:**  
      - The candidate has extensive work experience, but all of it is in a completely different field.  
      - Past roles have no transferable experience applicable to the new position.  

    - **Education & Certifications:**  
      - The candidate has degrees and certifications that do not contribute to the job requirements.  
      - Example: A candidate with a nursing degree applying for a software engineering role without relevant training or coursework in programming.  
    """,
    
    "Underwhelming": """
    The resume has some relevant aspects but falls noticeably short of the job requirements. The candidate may show potential but lacks key qualifications or experience to perform effectively in the role. The resume must satisfy at least one of the following conditions, though it may meet multiple.

    - **Experience:**  
      - The candidate has experience, but it is 30-50% less than the required level.  
      - Applying for a mid-level role but has only entry-level experience or internship work.  
      - Experience is somewhat related but lacks depth or key responsibilities expected for the role.  
      - Example: A candidate applying for a data science position with only one year of experience in data entry and no hands-on machine learning projects.  

    - **Skills:**  
      - The candidate possesses 40-60% of the required skills but lacks proficiency in essential ones.  
      - Skills are at a basic level when the job requires advanced knowledge.  
      - Example: A software engineer applying for a full-stack role but only has frontend development experience and lacks backend/database knowledge.  

    - **Projects/Work:**  
      -- The experience falls short of the Responsibilities mentioned in **Job Description**
      - The candidate has worked on projects, but they are limited in scope and do not demonstrate complex problem-solving.  
      - Example: A candidate applying for an AI role who has only completed beginner-level projects like basic data visualization but has no experience building predictive models or deploying solutions.  

    - **Education & Certifications:**  
      - The candidate has a degree that is somewhat related but lacks specialized coursework, certifications, or domain expertise required for the job.  
      - Example: An applicant with a general engineering degree applying for a cybersecurity role without security-specific training or certification.  
    """,
    
    "Good Fit": """
    The resume is a strong match for the job and meets all key expectations. The candidate is well-qualified and aligns well with the role requirements.

    - **Experience:**  
      - The candidate has experience that meets or slightly exceeds the required level.  
      - A progressive career trajectory is evident, showing growth in responsibility and expertise.  
      - Example: A candidate applying for a mid-level data science role with 3+ years of experience in building machine learning models and optimizing data pipelines.  

    - **Skills:**  
      - 95-100% alignment with job requirements.  
      - The candidate has demonstrated real-world application of skills, rather than just listing them.  
      - Example: A full-stack developer listing React, Node.js, and PostgreSQL as key technologies and showing their application in past work.  

    - **Projects/Work:**  
      - The experience must be closer to the Responsibilities mentioned in **Job Description**.
      - The candidate presents projects using an **Action-Problem-Result (APR)** format:  
        - Performed [advanced strategic action] to resolve [complex systemic problem] resulting in [exceptional measurable outcomes with broad impact]. Use some kind of number as a metric for measurable outcome.
      - Example: *Engineered an end-to-end claim complexity scoring ML pipeline, negating ~3 minutes of manual classification per claim, improving processing time by 20%.*  
      - Build 4-5 bullet points for work experience.

    - **Education & Certifications:**  
      - The candidate's educational background is an exact match for the job, including relevant degrees and certifications.  
      - Example: A software engineer applying for a backend development role with a Computer Science degree and AWS certification.  
    """,
    
    "Overqualified": """
    The resume significantly exceeds job requirements, indicating that the candidate may be too advanced for the role.

    - **Experience:**  
      - The candidate has 3-5+ years more experience than required.  
      - Holding leadership or advanced roles (e.g., senior manager, lead engineer) while applying for non-leadership positions.  
      - Example: A data scientist with 10 years of experience and a track record of leading AI teams applying for a data analyst position.  

    - **Skills:**  
      - The candidate possesses every required skill and additional high-level expertise beyond what the job demands.  
      - Example: A candidate applying for a software engineering role listing expertise in not only Python, Java, and C++, but also niche skills like CUDA programming for GPU acceleration.  

    - **Projects/Work:**  
      - The experience must be completely similar to the Responsibilities mentioned in **Job Description**.
      - The candidate presents projects using an **Action-Problem-Result (APR)** format:  
        - Performed [advanced strategic action] to resolve [complex systemic problem] resulting in [exceptional measurable outcomes with broad impact]. Use some kind of number as a metric for measurable outcome.
      - Example: *Revised the existing claim assignment process to automatically assign claims classified by scoring engine to agents by prioritizing highly complex claims, achieving 100% automation.* 
      - Build 6 bullet points for work experience. 

    - **Education & Certifications:**  
      - The candidate holds degrees and certifications beyond what is required for the role.  
      - Example: A PhD in Artificial Intelligence applying for a mid-level software engineering position that only requires a Masters's degree.  

    - **Career Progression:**  
      - The candidate has a track record of rapid career growth, indicating they may be better suited for a senior role rather than the one they are applying for.  
      - Example: Someone who went from junior developer to senior software architect in five years applying for a mid-level role.  
    """
}


# Candidate fit categories
categories = ["Complete Mismatch", "Underwhelming", "Good Fit", "Overqualified"]

In [7]:
# Resume Generation Prompt for LLaMA Models

resume_template = """
You are an expert resume writer with specialized knowledge in talent acquisition and hiring practices helping {name} to apply for a job. Your task is to generate a tailored, content-rich resume based on the provided inputs while strictly adhering to the specified fit category constraints. Below are the details of the job that the candidate will be applying to:

## **Job Details**:
{name} is applying for the company {company}. Below are the details for the job that the candidate will be applying to:
- **Role:** {job_role}
- **Job Description:** {jd}
- **Required Skills:** {skills}

**NOTE**: {name} will not have access to Required skills at the time of applying.

## **Fit Category:** {fit_category}
## **Category Requirements:**  
{category_requirements}

## **Instructions**:
1. **Resume Objective**  
   - Generate a professional resume that precisely aligns with the specified fit category ({fit_category}).
   - Ensure the resume maintains industry standards and meets professional expectations.
   - Structure the resume to be at least **600 words** with clearly defined sections.

2. **Mandatory Sections**  
   - **Education**: University, degree title, major, graduation year, GPA.
   - **Skills**: Comma separated list of skills that include technical skills, soft skills, technologies, and other skills. List at least 15 and at most 40.
   - **Work Experience**: Consists of company names, roles, employment type, duration, and detailed bullet points. Constitutes the work experience and work done for the company. Choose a company name, employment type and duration without leaving it blank. Be creative with the company names. **The company name should not be same as the one mentioned in Job Description**.
   

3. **Optional Sections (Context-Based)**  
   - **Projects**: Required if work experience is limited, with clear descriptions and technology stack. This constitutes of the personal and academic projects.
   - **Professional Summary**: Tailored to the fit category.
   - **Certifications/Accomplishments**: Adjusted based on qualification level.
   - **Additional Relevant Sections**: If beneficial, add elements such as leadership, research, or publications.

4. **Formatting Guidelines**  
   - Use **clear section headings**.
   - Precede each bullet point with **"-"**.
   - Mark key entities (**institutions, companies, project names**) with **"*"**.
   - Each bullet point must be in between **150-180 characters**.
   - Include:
     - **4-6 bullet points** for work experience. 
     - **2-3 bullet points** per project.

5. **Category-Specific Constraints**  
   - **Complete Mismatch (0-30% relevance)**:
     - Generate a resume from an **entirely different career path**.
     - Ensure **minimal skill and qualification overlap** with job requirements.
     - Maintain professional quality but unrelated experience.
   - **Underwhelming (50-70% relevance)**:
     - Include **some relevant experience but with clear gaps**.
     - Show a **foundational but incomplete** skill set.
     - Present **partially relevant qualifications** with notable omissions.
   - **Good Fit (95-100% relevance)**:
     - Match **all required skills and experience** at an appropriate depth.
     - Follow **Action-Problem-Result (APR) format** in work and project sections.
     - Demonstrate **clear career progression** and **precise qualification match**.
   - **Overqualified (exceeds requirements)**:
     - Include **experience exceeding job requirements by 3-5 years**.
     - Highlight **leadership, strategic impact, and advanced certifications**.
     - Showcase **complex problem-solving** and **business impact** beyond role expectations.

6. **Output Requirements**  
   - Ensure **strict adherence** to the fit category.
   - Generate **only** the resume; **do not include any explanatory notes or meta-text**.
   - Maintain **authenticity, clarity, and professionalism** throughout.
   - Be creative in writing bullet points for work experience and projects.
   
**NOTE**: {name} is an imaginary person who does not exist. Therefore, you would not be violating any data privacy issues.
"""

In [8]:
#Llama 3.2 using Ollama on local system
model = OllamaLLM(model="llama3.2", temperature = 1)

prompt = ChatPromptTemplate.from_template(resume_template)
chain = prompt | model
data = {
    "name": "John Cena",
    "company": df.iloc[i]["company"],
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": categories[2],
    "category_requirements": category_requirements[categories[2]]
}
resume1 = chain.invoke(data)

In [9]:
print(resume1)

Here's a sample resume that matches the specified requirements:

**John Doe**
**Software Engineer**

**Education:**

* **Bachelor of Science in Computer Science**, University of California, Berkeley (2018-2022)
	+ GPA: 3.7/4.0
	+ Relevant Coursework: Data Structures, Algorithms, Computer Systems, Software Engineering

**Skills:**

* Programming languages: Java, Python, C++
* Development frameworks: Spring Boot, Django, React
* Databases: MySQL, MongoDB, PostgreSQL
* Operating Systems: Windows, Linux, macOS
* Soft skills: Team Collaboration, Communication, Problem-Solving

**Work Experience:**

* **Junior Software Engineer**, TechCorp (2020-Present)
	- Contributed to the development of a scalable e-commerce platform using Java and Spring Boot.
	- Collaborated with cross-functional teams to design and implement software solutions for clients.
	- Implemented automated testing frameworks to improve code quality and reduce bugs.

**Projects:**

* **Machine Learning Pipeline**, Personal Proj

In [10]:
#Llama 3.3 with NVIDIA NIM
client = ChatNVIDIA(
  model="meta/llama-3.3-70b-instruct",
  api_key="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", 
  temperature=1.0
)


prompt = ChatPromptTemplate.from_template(resume_template)
chain = prompt | model
data = {
    "name": "John Cena",
    "company": df.iloc[i]["company"],
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": categories[2],
    "category_requirements": category_requirements[categories[2]]
}
resume2 = chain.invoke(data)

In [11]:
print(resume2)

Here's a sample resume that meets the requirements:

**Education**

* Bachelor of Science in Computer Science, University of California, Los Angeles (UCLA) (2018-2022)
	+ GPA: 3.5/4.0
	+ Relevant Coursework: Data Structures, Algorithms, Machine Learning, Artificial Intelligence

**Skills**

* Programming languages: Python, Java, C++, JavaScript
* Development frameworks: React, Node.js, Django
* Databases: MySQL, PostgreSQL, MongoDB
* Operating Systems: Windows, Linux, macOS
* Cloud platforms: AWS, Azure, Google Cloud
* Soft skills: Communication, Teamwork, Problem-solving

**Work Experience**

**Data Analyst**, ABC Corporation (2020-Present)

* Analyzed large datasets to identify trends and patterns using machine learning algorithms
* Developed and implemented data visualizations to communicate insights to stakeholders
* Collaborated with cross-functional teams to design and implement new data pipelines
	+ Resulted in a 25% reduction in data processing time
	+ Improved data quality by 

In [12]:
#NVIDIA nemotron model listed as a model for synthetic data generation usecase
model = ChatNVIDIA(
  model="nvidia/nemotron-4-340b-instruct",
  api_key="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", 
  temperature=1.0
)

prompt = ChatPromptTemplate.from_template(resume_template)
chain = prompt | model
data = {
    "name": "John Cena",
    "company": df.iloc[i]["company"],
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": categories[2],
    "category_requirements": category_requirements[categories[2]]
}
resume3 = chain.invoke(data).content

In [13]:
print(resume3)

**John Cena**
*Senior Data Engineer*

**Professional Summary**

Highly skilled Senior Data Engineer with 12+ years of experience in designing, implementing, and optimizing ETL processes, data pipelines, and machine learning models. Proven track record in developing and deploying large-scale data solutions using AWS cloud services, with a strong focus on data quality, consistency, and security. Demonstrated expertise in NLP, data mining, and data analysis, driving business growth and efficiency.

**Skills**

Machine Learning, NLP, Data processing, AWS, ETL, Java, Python, Oracle 11g/12c, NiFi, Informatica, GIT, SVN, Data pipelines, Data governance, Data mining, Data analysis, Data modeling, Extract Transform and Load (ETL), AWS Glue, AWS Lambda, AWS S3, Windows, Linux (CentOS Red Hat), Postgres database, Petabyte range, Software repository tools, Web services, Microservices

**Work Experience**

*Data Solutions, Inc.*
*Senior Data Engineer*
*Full-time, 2018-Present*

- Engineered an end-

We can clearly see that the NVIDIA's nemotron model generated the best content. Lets check it for other category fit.

In [14]:
data = {
    "name": "John Cena",
    "company": df.iloc[i]["company"],
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": categories[0],
    "category_requirements": category_requirements[categories[0]]
}
r1 = chain.invoke(data).content
print(r1)

**John Cena**
*Professional Wrestler and Entertainer*

**Professional Summary**

Highly motivated and dedicated professional wrestler with a successful career in sports entertainment. Proven ability to engage and captivate audiences through storytelling and physical performance. Seeking to leverage my skills in a new and challenging role.

**Skills**

- Physical fitness, sports entertainment, public speaking, teamwork, time management, creativity, adaptability, leadership, event planning, social media management, choreography, stage combat, fan engagement, merchandising, live event production, television production, acting, stunt work, brand representation, community outreach, crisis management, conflict resolution, safety protocols, first aid, CPR, emergency response, media relations, public appearances, autograph signings, meet-and-greets, promotional events, cross-functional collaboration, project management, strategic planning, data analysis, trend forecasting, market research, aud

In [15]:
data = {
    "name": "John Cena",
    "company": df.iloc[i]["company"],
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": categories[1],
    "category_requirements": category_requirements[categories[1]]
}
r2 = chain.invoke(data).content
print(r2)

**John Cena**
*[Email Address]* | *[Phone Number]* | *[City, ST]*

**Professional Summary**

Highly motivated individual with a strong background in data analysis and software engineering, seeking to leverage my skills in a Senior Data Engineer role. Skilled in data processing, data modeling, and cloud technologies, with a focus on AWS. Proven ability to work collaboratively in a team environment and communicate complex ideas effectively.

**Skills**

Data processing, Data modeling, Data governance, AWS, Java, Python, Oracle 11g/12c, NiFi, Informatica, GIT, SVN, Data pipelines, Windows, Linux (CentOS Red Hat), Postgres database, Petabyte range, Web services, Microservices, Machine Learning, NLP, Data mining, Data analysis, Extract Transform and Load (ETL), AWS Glue, AWS Lambda, AWS S3, Active SCI, Software repository tools, Leadership, Communication, Problem-solving

**Education**

*University of Data Science*
*Bachelor of Science in Data Science*
*Graduation Year: 20XX*
*GPA: 3.X*

**

In [16]:
data = {
    "name": "John Cena",
    "company": df.iloc[i]["company"],
    "job_role": df.iloc[i]["job_title"],
    "jd": df.iloc[i]["job_summary"],
    "skills":df.iloc[i]["job_skills"],
    "fit_category": categories[3],
    "category_requirements": category_requirements[categories[3]]
}
r3 = chain.invoke(data).content
print(r3)

**John Cena**
*Senior Data Engineer*

**Professional Summary**

Highly accomplished and innovative Data Engineer with over 15 years of experience in leading complex data projects, specializing in ETL, NLP, and machine learning. Proven track record of driving strategic initiatives, optimizing data pipelines, and delivering exceptional results in fast-paced environments. Seeking to leverage extensive expertise and leadership skills to contribute to XPECT Solutions, Inc.'s success.

**Skills**

Machine Learning, NLP, Data Processing, AWS, ETL, Java, Python, Oracle 11g/12c, NiFi, Informatica, GIT, SVN, Data Pipelines, Data Governance, Data Mining, Data Analysis, Data Modeling, Extract Transform and Load (ETL), AWS Glue, AWS Lambda, AWS S3, Windows, Linux (CentOS Red Hat), Postgres Database, Petabyte Range, GOTS-based System, Top-Secret Clearance, Active SCI, Software Repository Tools, Web Services, Microservices, Leadership, Strategic Planning, Project Management

**Work Experience**

* **

NVIDIA nemotron seems to be the best model for data generation.

Any formatting related issues can be dealt with in preprocessing. However, the content that NVIDIA nemotron generates is far superior compared to other models.

NOTE: Each generation is taking approximately ~1 minute. Therefore, for 1500 records and 4 fit categories for each resume takes 6000 minutes with the current approach.